In [ ]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

# !ls "/content/drive/My Drive"
!cp '/content/drive/My Drive/data/nus-cs5242.zip' nus-cs5242.zip
# !cp '/content/drive/My Drive/AI/model_output_xception_overfitting-20201014T084949Z-001.zip' trained_model.zip
!unzip -qq nus-cs5242.zip
# !unzip -qq trained_model.zip 


Mounted at /content/drive


In [ ]:
!ls -l "drive"

total 4
drwx------ 3 root root 4096 Oct 16 06:12 'My Drive'


In [ ]:
!cp '/content/drive/My Drive/AI/model_output_xception_overfitting-20201014T084949Z-001.zip' trained_model.zip
!unzip -qq trained_model.zip 

In [ ]:
import numpy as np
import pandas as pd
import cv2
import skimage as sk
from sklearn.model_selection import KFold
from sklearn.utils import resample
from keras.preprocessing import image




import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import BatchNormalization , Activation, MaxPooling2D, Dense, Dropout, Flatten
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator # new!
from matplotlib import pyplot as plt
from tensorflow.keras.applications import ResNet50V2, Xception, InceptionV3
from tensorflow.keras.initializers import Zeros, glorot_normal
from tensorflow.keras.callbacks import TensorBoard ,ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
import os

In [ ]:
#parameter

BATCH_SIZE=32
IMG_SIZE=(512, 512)
w_init = glorot_normal()
b_init = Zeros()
data_train_label = "train_label.csv"
data_train_dir = "train_image/train_image/"
data_test_dir = "test_image/"
output_dir = 'model_output/'
output_training= output_dir + 'check_point'
output_logs = output_dir + 'logs'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
if not os.path.exists(output_training):
    os.makedirs(output_training)
if not os.path.exists(output_logs):
    os.makedirs(output_logs)

In [ ]:
# images = []
# for img in os.listdir(data_train_dir):
#     print(img)
#     img = os.path.join(data_train_dir, img)
#     img = image.load_img(img, target_size=IMG_SIZE)
#     img = image.img_to_array(img)
#     img = np.expand_dims(img, axis=0)
#     img = img/255.0
#     images.append(img)
# images = np.vstack(images)

# images_2 = images

In [ ]:
print(images[0])

In [ ]:
# modelcheckpoint_acc = ModelCheckpoint(filepath=output_dir + "/nf6/xception_acc_{epoch:03d}-{val_loss:.3f}-{val_accuracy:.3f}.h5", 
#                                   monitor='val_accuracy',
#                                   mode = 'max',
#                                   save_best_only=True)

# modelcheckpoint_loss = ModelCheckpoint(filepath=output_dir + "/nf6/xception_loss_{epoch:03d}-{val_loss:.3f}-{val_accuracy:.3f}.h5", 
#                                   monitor='val_loss',
#                                   mode = 'min',
#                                   save_best_only=True)

tensorboard = TensorBoard(output_logs)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              mode='min',verbose = 1,
                              patience=5, min_lr=0.00001)

In [ ]:
train_label = pd.read_csv(data_train_label)
filename = train_label.ID.map(lambda x: str(x)+".png").to_numpy()
# filename = train_label.ID.to_numpy()
label = train_label.Label.map(lambda x: str(x)).to_numpy()
train_label = train_label.assign(ID=filename, Label=label)
print(train_label.shape)
print(train_label.head())


(1164, 2)
      ID Label
0  0.png     1
1  1.png     0
2  2.png     1
3  3.png     1
4  4.png     1


In [ ]:
def upsampling(data_frame):
  l0 = data_frame[data_frame.Label == '0']
  l1 = data_frame[data_frame.Label == '1']
  l2 = data_frame[data_frame.Label == '2']

  n_l0 = l0.shape[0]
  n_l1 = l1.shape[0]
  n_l2 = l2.shape[0]
  max_v = np.max([n_l0, n_l1, n_l2])
  print("Before upsampling: ")
  print("Label 0 : ", n_l0)
  print("Label 1 : ", n_l1)
  print("Label 2 : ", n_l2)
  print("n_max : ", max_v)

  if n_l0 < max_v:
    print("Need to upsample 0")
    n_rand = max_v - n_l0
    choices = None
    if n_rand > n_l0:
      choices = np.random.choice(n_l0, n_rand)
    else:
      choices = np.random.choice(n_l0, n_rand)

    l0_sample = l0.iloc[choices]
    l0 = l0.append(l0_sample,ignore_index=True)
    l0 = l0.sample(frac=1)
    # print(l0.ID.value_counts()



  if n_l1 < max_v:
    print("Need to upsample 1")
    n_rand = max_v - n_l1
    choices = None
    if n_rand > n_l1:
      choices = np.random.choice(n_l1, n_rand)
    else:
      choices = np.random.choice(n_l1, n_rand)

    l1_sample = l1.iloc[choices]
    l1 = l1.append(l1_sample,ignore_index=True)
    l1 = l1.sample(frac=1)
    # print(l1.ID.value_counts()


  if n_l2 < max_v:
    print("Need to upsample 2")
    n_rand = max_v - n_l2
    choices = None
    if n_rand > n_l2:
      choices = np.random.choice(n_l2, n_rand)
    else:
      choices = np.random.choice(n_l2, n_rand)

    l2_sample = l2.iloc[choices]
    l2 = l2.append(l2_sample,ignore_index=True)
    l2 = l2.sample(frac=1)
    # print(l0.ID.value_counts()


  # print(data_a[2].ID.value_counts())

  df_upsampled = pd.concat([l0, l1, l2])
  df_upsampled = df_upsampled.sample(frac=1).reset_index(drop=True)

  print("After upsampling:")
  print("Label 0 : ", df_upsampled[df_upsampled.Label == '0'].shape[0])
  print("Label 1 : ", df_upsampled[df_upsampled.Label == '1'].shape[0])
  print("Label 2 : ", df_upsampled[df_upsampled.Label == '2'].shape[0])
  # print(df_upsampled.Label.value_counts())
  return df_upsampled

upsampling(train_label)


Before upsampling: 
Label 0 :  312
Label 1 :  538
Label 2 :  314
n_max :  538
Need to upsample 0
Need to upsample 2
After upsampling:
Label 0 :  538
Label 1 :  538
Label 2 :  538


,ID,Label
0,1082.png,0
1,386.png,1
2,245.png,1
3,859.png,1
4,610.png,1
...,...,...
1609,688.png,0
1610,398.png,2
1611,169.png,0
1612,905.png,2


In [ ]:
testgen=ImageDataGenerator(rescale=1./255)

data_test=testgen.flow_from_directory(directory=data_test_dir, 
                                      shuffle = False, target_size=IMG_SIZE, class_mode='categorical', batch_size=1)

Found 292 images belonging to 1 classes.


In [ ]:
res_netv2 = ResNet50V2(input_shape=(512,512,3), include_top=False, weights="imagenet", pooling="avg")
for layer in res_netv2.layers:
    layer.trainable = False

model = Sequential()
model.add(res_netv2)
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(276, activation='relu', kernel_initializer=w_init, bias_initializer=b_init, kernel_regularizer='l2'))
model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax', kernel_initializer=w_init, bias_initializer=b_init, kernel_regularizer='l2'))
print("using gobal AVG pooling")

94674944/94668760 [==============================] - 3s 0us/step
using gobal AVG pooling


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Functional)      (None, 2048)              23564800  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1049088   
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 1539      
Total params: 24,617,475
Trainable params: 1,051,651
Non-trainable params: 23,565,824
____________________________________

In [ ]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.00001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-08,
    amsgrad=False,
    name="Adam",
)


loss = tf.keras.losses.CategoricalCrossentropy(
    from_logits=False,
    label_smoothing=0.05,
    reduction="auto",
    name="categorical_crossentropy",
)



In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
print(tf.keras.backend.eval(model_avg.optimizer.lr))

1e-05


In [ ]:
kf = KFold(n_splits=6, shuffle = True, random_state = 60)

In [ ]:

train_gen=ImageDataGenerator(rescale=1./255)

val_gen=ImageDataGenerator(rescale=1./255)

kfold_num = 1
for train_index, val_index in kf.split(train_label):
    print("==============================Fold number = ", kfold_num)
    # modelcheckpoint_loss = ModelCheckpoint(filepath=output_dir + "/resnet_" + str(kfold_num) + ".h5", 
    #                                                         monitor='val_loss',
    #                                                         mode = 'min',
    #                                                         save_best_only=True)
    

    # es = EarlyStopping(monitor='val_loss', mode='min', patience = 7, restore_best_weights = True, verbose=1)

    train_set = train_label.loc[train_index]
    val_set = train_label.loc[val_index]
    train_set_up = upsampling(train_set)

    train_data = train_gen.flow_from_dataframe(dataframe=train_set_up, directory=data_train_dir,
                                                      x_col="ID", y_col="Label", shuffle = True, class_mode="categorical", seed=123,
                                                      target_size=IMG_SIZE, batch_size=BATCH_SIZE)
    val_data = val_gen.flow_from_dataframe(dataframe=val_set, directory=data_train_dir,
                                                      x_col="ID", y_col="Label", shuffle = True, class_mode="categorical", seed=123,
                                                      target_size=IMG_SIZE, batch_size=BATCH_SIZE)
    
    model.fit(train_data, epochs=50, validation_data=val_data, verbose=1,callbacks=[tensorboard])
    # model = tf.keras.models.load_model(filepath=output_dir + "/resnet_" + str(kfold_num) + ".h5")
    kfold_num += 1
    model.evaluate(val_data, verbose=2)




==============================Fold number =  1
Before upsampling: 
Label 0 :  261
Label 1 :  453
Label 2 :  256
n_max :  453
Need to upsample 0
Need to upsample 2
After upsampling:
Label 0 :  453
Label 1 :  453
Label 2 :  453
Found 1359 validated image filenames belonging to 3 classes.
Found 194 validated image filenames belonging to 3 classes.
Epoch 1/50
43/43 [==============================] - 20s 455ms/step - loss: 6.1969 - accuracy: 0.4297 - val_loss: 5.9298 - val_accuracy: 0.5103
Epoch 2/50
43/43 [==============================] - 18s 428ms/step - loss: 5.9203 - accuracy: 0.5305 - val_loss: 5.7649 - val_accuracy: 0.6392
Epoch 3/50
43/43 [==============================] - 18s 424ms/step - loss: 5.7554 - accuracy: 0.6115 - val_loss: 5.6149 - val_accuracy: 0.6753
Epoch 4/50
43/43 [==============================] - 19s 432ms/step - loss: 5.5707 - accuracy: 0.6954 - val_loss: 5.4901 - val_accuracy: 0.7165
Epoch 5/50
43/43 [==============================] - 18s 426ms/step - loss: 5.5144

In [ ]:
model.save("resnet50_276_cv_up6_e50_lr00001.h5")

In [ ]:

eval_gen=ImageDataGenerator(rescale=1./255)

kfold_num = 1
for train_index, val_index in kf.split(train_label):
    print("==============================Fold number = ", kfold_num)
    kfold_num += 1

    val_set = train_label.loc[val_index]
    val_data = eval_gen.flow_from_dataframe(dataframe=val_set, directory=data_train_dir,
                                                      x_col="ID", y_col="Label", shuffle = True, class_mode="categorical", seed=123,
                                                      target_size=IMG_SIZE, batch_size=BATCH_SIZE)
    
    model.evaluate(val_data, verbose=2)

==============================Fold number =  1
Found 194 validated image filenames belonging to 3 classes.
7/7 - 2s - loss: 0.8850 - accuracy: 1.0000
==============================Fold number =  2
Found 194 validated image filenames belonging to 3 classes.
7/7 - 2s - loss: 0.8855 - accuracy: 1.0000
==============================Fold number =  3
Found 194 validated image filenames belonging to 3 classes.
7/7 - 2s - loss: 0.8868 - accuracy: 1.0000
==============================Fold number =  4
Found 194 validated image filenames belonging to 3 classes.
7/7 - 2s - loss: 0.8878 - accuracy: 1.0000
==============================Fold number =  5
Found 194 validated image filenames belonging to 3 classes.
7/7 - 2s - loss: 0.8881 - accuracy: 1.0000
==============================Fold number =  6
Found 194 validated image filenames belonging to 3 classes.
7/7 - 2s - loss: 0.9091 - accuracy: 0.9948


In [ ]:
model_max.save("model_max_2.h5")

!cp -r 'model_max_2.h5' '/content/drive/My Drive/data/model_max_2.h5' 

In [ ]:
predicted_classes = np.argmax(model.predict(data_test), axis = -1)

In [ ]:
test_id = data_test.filenames
new_test_id = [x.replace('.png', '').replace('test_image/', '') for x in test_id]


In [ ]:
evaluation = pd.DataFrame({'ID':new_test_id, 'Label':predicted_classes})
evaluation.to_csv(output_dir + "ensemble_resnet_inception_xception.csv", index=False) # Public Score: 0.76555
evaluation.head()

,ID,Label
0,0,0
1,1,1
2,10,0
3,100,2
4,101,1
